In [4]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())  # Check number of GPUs available


True
1


In [8]:
import model_loader
import pipeline
from PIL import Image 
from pathlib import Path
from transformers import CLIPTokenizer
import torch


In [9]:
DEVICE = "cuda"

In [10]:
ALLOW_CUDA = True
ALLOW_MPS = False

In [11]:
if torch.cuda.is_available() and ALLOW_CUDA:
    DEVICE = "cuda"
elif (torch.has_mps or torch.backends.mps.is_available()) and ALLOW_MPS:
    DEVICE = "mps"
print(f"Using device: {DEVICE}")

Using device: cuda


In [12]:
import os
import requests

# Define URLs for the tokenizer and model files
vocab_url = "https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/resolve/main/tokenizer/vocab.json"
merges_url = "https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/resolve/main/tokenizer/merges.txt"
model_url = "https://huggingface.co/stable-diffusion-v1-5/stable-diffusion-v1-5/resolve/main/v1-5-pruned-emaonly.ckpt"

# Step 1: Create a folder structure to store the downloaded files
# You can customize the folder name and location if needed
folder_name = "model"
os.makedirs(folder_name, exist_ok=True)

# Step 2: Function to download a file and save it
def download_file(url, destination_path):
    response = requests.get(url)
    if response.status_code == 200:
        with open(destination_path, 'wb') as file:
            file.write(response.content)
        print(f"Downloaded: {destination_path}")
    else:
        print(f"Failed to download {url}")

# Step 3: Download the tokenizer files (vocab.json and merges.txt)
download_file(vocab_url, os.path.join(folder_name, "vocab.json"))
download_file(merges_url, os.path.join(folder_name, "merges.txt"))

# Step 4: Download the model checkpoint file (v1-5-pruned-emaonly.ckpt)
download_file(model_url, os.path.join(folder_name, "v1-5-pruned-emaonly.ckpt"))

# Step 5: Verify the files are downloaded in the folder
print("\nDownloaded files are stored in the 'model' folder.")


Downloaded: model\vocab.json
Downloaded: model\merges.txt
Downloaded: model\v1-5-pruned-emaonly.ckpt

Downloaded files are stored in the 'model' folder.


In [17]:
pip install pytorch_lightning

   ---------------------------------------- 0.0/815.2 kB ? eta -:--:--
   ------------------------------- ------- 655.4/815.2 kB 20.8 MB/s eta 0:00:01
   --------------------------------------- 815.2/815.2 kB 26.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/926.4 kB ? eta -:--:--
   ---------------------------------------- 926.4/926.4 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [18]:
import model_loader
import pipeline
from PIL import Image
from pathlib import Path
from transformers import CLIPTokenizer
import torch

DEVICE = "cuda"

ALLOW_CUDA = False
ALLOW_MPS = False

if torch.cuda.is_available() and ALLOW_CUDA:
    DEVICE = "cuda"
elif (torch.has_mps or torch.backends.mps.is_available()) and ALLOW_MPS:
    DEVICE = "mps"
print(f"Using device: {DEVICE}")

tokenizer = CLIPTokenizer(r"C:\Users\Deepak\Desktop\Image generation  Project\model\vocab.json", merges_file=r"C:\Users\Deepak\Desktop\Image generation  Project\model\merges.txt")
model_file = r"C:\Users\Deepak\Desktop\Image generation  Project\model\v1-5-pruned-emaonly.ckpt"
models = model_loader.preload_models_from_standard_weights(model_file, DEVICE)

## TEXT TO IMAGE

# prompt = "A dog with sunglasses, wearing comfy hat, looking at camera, highly detailed, ultra sharp, cinematic, 100mm lens, 8k resolution."
prompt = "nuclear bomb in usa "
uncond_prompt = ""  # Also known as negative prompt
do_cfg = True
cfg_scale = 8  # min: 1, max: 14

## IMAGE TO IMAGE

input_image = None
# Comment to disable image to image
image_path = "../images/dog.jpg"
# input_image = Image.open(image_path)
# Higher values means more noise will be added to the input image, so the result will further from the input image.
# Lower values means less noise is added to the input image, so output will be closer to the input image.
strength = 0.9

## SAMPLER

sampler = "ddpm"
num_inference_steps = 50
seed = 42

output_image = pipeline.generate(
    prompt=prompt,
    uncond_prompt=uncond_prompt,
    input_image=input_image,
    strength=strength,
    do_cfg=do_cfg,
    cfg_scale=cfg_scale,
    sampler_name=sampler,
    n_inference_steps=num_inference_steps,
    seed=seed,
    models=models,
    device=DEVICE,
    idle_device="cpu",
    tokenizer=tokenizer,
)

# Combine the input image and the output image into a single image.
Image.fromarray(output_image)

C:\Users\Deepak\AppData\Local\Temp\ipykernel_15736\3329725703.py:15: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  elif (torch.has_mps or torch.backends.mps.is_available()) and ALLOW_MPS:


Using device: cuda


OutOfMemoryError: CUDA out of memory. Tried to allocate 114.00 MiB. GPU 0 has a total capacity of 4.00 GiB of which 0 bytes is free. Of the allocated memory 10.53 GiB is allocated by PyTorch, and 232.92 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [20]:
!nvidia-smi


Mon Nov 25 08:48:09 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 561.17                 Driver Version: 561.17         CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                  Driver-Model | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 2050      WDDM  |   00000000:01:00.0 Off |                  N/A |
| N/A   45C    P8              5W /   30W |    3945MiB /   4096MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----